# Recommender System using AutoEncoder

### Data Preprocessing

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from data_prepocess import load_data
from torch.utils.data import DataLoader

DATASET = '../movielens-dataset/ratings.csv'

training_set, test_set, n_users, n_movies = load_data(DATASET)

### Build Model

In [2]:
# Define the neural network
import torch
import torch.nn as nn
import torch.optim as optim

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(n_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, n_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
model = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr = 0.01, weight_decay = 0.5)

### Train Model

In [ ]:
from torch.autograd import Variable
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

if train_on_gpu:
    model.cuda()

n_epoch = 200
for epoch in range(1, n_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(n_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        
        if train_on_gpu:
            input, target = input.cuda(), target.cuda()
        
        if torch.sum(target.data > 0) > 0:
            output = model(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = n_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))